In [10]:
import sqlite3
import pandas as pd
db_file = "Fund.db"
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'FB', 'TSLA', 'NVDA', 'PYPL', 'ADBE', 'INTC']#tickers contenus dans MAIN_DATA_FRAME

start_date = "02/01/2023"#toute la periode start date (pour tout telecharger dun coup)
end_date = "31/12/2024"#pareil

## 1. Creation et peuplement de la base

In [11]:
from base_builder import DatabaseBuilder

In [12]:
db_manager = DatabaseBuilder(db_file)
db_manager.create_tables()
db_manager.insert_clients_data(3)
db_manager.insert_managers_data(3)

Tables created successfully.
3 clients insérés avec succès dans la table 'Clients'.
3 managers insérés avec succès dans la table 'Managers'.


## 2. Collecte et traitement des datas

In [13]:
from data_collector import GetData

In [14]:
gdata = GetData(tickers, start_date, end_date)
gdata.main_data_frame()#on cree le dataframe avec TOUS les actifs TOUTE la periode
#il manquerait le traitement des datas

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['FB']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,IMPORT_DATE,PRICE,TICKER,SECTOR
0,2023-01-03,123.632523,AAPL,Technology
1,2023-01-04,124.907692,AAPL,Technology
2,2023-01-05,123.583099,AAPL,Technology
3,2023-01-06,128.130249,AAPL,Technology
4,2023-01-09,128.654129,AAPL,Technology
...,...,...,...,...
4504,2024-12-23,20.200001,INTC,Technology
4505,2024-12-24,20.400000,INTC,Technology
4506,2024-12-26,20.440001,INTC,Technology
4507,2024-12-27,20.299999,INTC,Technology


In [15]:
from base_update import BaseUpdate 
sub_start_date = "01/01/2023"#chaque semaine, les strat lancent la fct et definissent les sub périodes
#de cette maniere, products ne contient que la data jusqu'à la date t (no forward looking sur nos strats)
sub_end_date = "31/12/2023"

In [16]:
B_udt = BaseUpdate(tickers, sub_start_date, sub_end_date)
B_udt.update_products(db_file)#on met a jour products jusqu'à la data t

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['FB']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Successfully inserted 2250 new records into Products table for the period 01/01/2023 to 31/12/2023


In [17]:
from base_update import update_portefeuille1
update_portefeuille1(db_file,['AAPL', 'MSFT'],'HY_EQUITY',50)


Portfolios mis à jour avecc succés


In [18]:
from strategies import RunAllStrat
start_date2="31/12/2023"
end_date2="15/01/2024"
b=RunAllStrat(db_file,start_date2,end_date2,tickers)
b.run()

[*********************100%***********************]  1 of 1 completed


Début de la simulation historique du 31/12/2023 au 15/01/2024

=== Simulation pour le lundi 01/01/2024 ===
Exécution de la stratégie pour la date historique : 01/01/2024
Updating products for historical week: 25/12/2023 to 31/12/2023



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['FB']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Successfully inserted 36 new records into Products table for the period 25/12/2023 to 31/12/2023
Running strategies...
Running HY_EQUITY strategy...
✓ Ticker trouvé : AAPL avec Portfolio ID 3, Manager ID 1
→ Insertion validée : Buy 18 x AAPL @ 191.38095092773438 le 2024-01-01
✓ Portfolio mis à jour pour AAPL: nouvelle quantité ajustée de 18
✓ Ticker trouvé : MSFT avec Portfolio ID 4, Manager ID 1
→ Insertion validée : Buy 28 x MSFT @ 372.50201416015625 le 2024-01-01
✓ Portfolio mis à jour pour MSFT: nouvelle quantité ajustée de 28

=== Simulation pour le lundi 08/01/2024 ===
Exécution de la stratégie pour la date historique : 08/01/2024
Updating products for historical week: 01/01/2024 to 07/01/2024



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['FB']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Successfully inserted 36 new records into Products table for the period 01/01/2024 to 07/01/2024
Running strategies...
Running HY_EQUITY strategy...
✓ Ticker trouvé : AAPL avec Portfolio ID 3, Manager ID 1
→ Insertion validée : Buy 25 x AAPL @ 191.38095092773438 le 2024-01-08
✓ Portfolio mis à jour pour AAPL: nouvelle quantité ajustée de 25
✓ Ticker trouvé : MSFT avec Portfolio ID 4, Manager ID 1
→ Insertion validée : Buy 43 x MSFT @ 372.50201416015625 le 2024-01-08
✓ Portfolio mis à jour pour MSFT: nouvelle quantité ajustée de 43

=== Simulation pour le lundi 15/01/2024 ===
Exécution de la stratégie pour la date historique : 15/01/2024
Updating products for historical week: 08/01/2024 to 14/01/2024



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['FB']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Successfully inserted 45 new records into Products table for the period 08/01/2024 to 14/01/2024
Running strategies...
Running HY_EQUITY strategy...
✓ Ticker trouvé : AAPL avec Portfolio ID 3, Manager ID 1
→ Insertion validée : Buy 34 x AAPL @ 191.38095092773438 le 2024-01-15
✓ Portfolio mis à jour pour AAPL: nouvelle quantité ajustée de 34
✓ Ticker trouvé : MSFT avec Portfolio ID 4, Manager ID 1
→ Insertion validée : Buy 68 x MSFT @ 372.50201416015625 le 2024-01-15
✓ Portfolio mis à jour pour MSFT: nouvelle quantité ajustée de 68

Simulation terminée !
Nombre de lundis simulés : 3
Période couverte : 31/12/2023 au 15/01/2024
